In [1]:
#importing libraries
import numpy as np
import pandas as pd
import math
from sklearn import datasets
from sklearn.model_selection import train_test_split
import pydot
from pprint import pprint

In [2]:
#node class an object of which represents a node of the tree
class Node:
    def __init__(self, split_on_feature, current_output):
        #contains the value of the feature upon which the node is goin to be split, none indicates a leaf node
        self.split_on_feature = split_on_feature
        
        #contains children nodes as a dictionary, key is the value of the feature upon which the node is split and value is the child node
        self.children = {}
        
        #contains the class with current majority in the node
        self.current_output = current_output
        
    def add_child(self, feature_value, node):
        self.children[feature_value] = node

In [3]:
#the following implements the Decision Tree class
class DecisionTreeClassifier:
    def _init_(self):
        self.root = None
        
    #counts the number of occurences of each class in the current subset of original data points
    def frequency_counter(self, Y):
        frequency = {}
        for cls in Y:
            if cls in frequency:
                frequency[cls] += 1
            else:
                frequency[cls] = 1
        return frequency
                
    #calculates the entropy/information gain for a single node
    def entropy(self, Y):
        frequency = self.frequency_counter(Y)
        info = 0
        total_instances = len(Y)
        
        for cls in frequency:
            pi = frequency[cls]/total_instances
            info += -(pi*math.log2(pi))
            
        return info
    
    #calculates the gain ratio achieved when a node is split upon a particular feature
    def gain_ratio(self, X, Y, split_on_feature):
        
        entropy_before_splitting = self.entropy(Y)
        entropy_after_splitting = 0
        split_info = 0
        
        split_on_feature_values = set(X[:, split_on_feature])
        
        df = pd.DataFrame(X)
        df[df.shape[1]] = Y
        
        total_instances = len(df)
        for value in split_on_feature_values:
            dfi = df[df[split_on_feature] == value]
            current_instances = len(dfi)
            entropy_after_splitting += ((current_instances/total_instances)*self.entropy(dfi.iloc[:, -1]))
            split_info += (-((current_instances/total_instances) * math.log2(current_instances/total_instances)))
            
        if split_info == 0:
            return math.inf
        
        info_gain = entropy_before_splitting - entropy_after_splitting
        gain_ratio = (info_gain/split_info)
        return gain_ratio
    
    #creates the decision tree recursively
    def decision_tree(self, X, Y, features_left, classes, current_level, all_features):
        
        print('Level', current_level)
        frequency = self.frequency_counter(Y)
        
        #############################################################################################################
        
        #base case - 1 when no features are left to split upon
        if(len(features_left) == 0):
            output = None
            max_frequency = -math.inf
            for cls in classes:
                if cls not in frequency:
                    print('Count of', cls, "=", 0)
                else:
                    if frequency[cls] > max_frequency:
                        output = cls
                        max_frequency = frequency[cls]
                    print('Count of', cls, "=", frequency[cls])
            
            print('Current Entropy is =', self.entropy(Y))          
            print('Reached leaf Node')
            print()
            return Node(None, output)
        
        #############################################################################################################
        
        #base case - 2 when we have reached a leaf node
        if (len(set(Y)) == 1):
            output = None
            for cls in classes:
                if cls in Y:
                    output = cls
                    print('Count of', cls, '=', len(Y))
                else :
                    print('Count of', cls, '=', 0)
            
            print('Current Entropy is =  0.0')
            print('Reached leaf Node')
            print()
            return Node(None, output)

        #############################################################################################################
        
        #calculating best feature to split upon i.e. feature with maximum gain ratio
        max_gain_ratio = -math.inf
        final_feature_to_split_upon = None
        for i in features_left :
            current_gain_ratio = self.gain_ratio(X, Y, i)
            if current_gain_ratio > max_gain_ratio:
                max_gain_ratio = current_gain_ratio
                final_feature_to_split_upon = i

        output = None
        max_frequency = -math.inf

        for cls in classes:
            if cls not in frequency:
                print('Count of', cls, '=', 0)
            else:
                if frequency[cls] > max_frequency:
                    output = cls
                    max_frequency = frequency[cls]
                print('Count of', cls, '=', frequency[cls])
     
        print('Current Entropy is =', self.entropy(Y))
        print('Splitting on feature' , all_features[final_feature_to_split_upon] , 'with gain ratio' , max_gain_ratio)
        print()
        
        #############################################################################################################
            
        #splitting and calling on split recursively for every value of chosen feature
        feature_values = set(X[:, final_feature_to_split_upon])
        df = pd.DataFrame(X)
        df[df.shape[1]] = Y

        current_node = Node(final_feature_to_split_upon, output)

        index = features_left.index(final_feature_to_split_upon)
        features_left.remove(final_feature_to_split_upon)
        
        for value in feature_values:
            dfi = df[df[final_feature_to_split_upon] == value]
            
            node = self.decision_tree(dfi.iloc[:, 0:-1].values, dfi.iloc[:, -1].values, features_left, classes, current_level + 1, all_features)
            current_node.add_child(value, node)
    
        features_left.insert(index, final_feature_to_split_upon)
        return current_node
    
    #function analogous to fit in scikit-learn
    def fit(self, X, Y):
        all_features = np.array([feature for feature in df.columns])
        features = [i for i in range(len(X[0]))]
        classes = set(Y)
        level = 0
        self.root = self.decision_tree(X, Y, features, classes, level, all_features)
        
    #function used to predict target value for a single data point 
    def predict_data_point(self, data, node):
        if len(node.children) == 0:
            return node.current_output

        value = data[node.split_on_feature]       
        if value not in node.children:
            return node.current_output
        
        return self.predict_data_point(data, node.children[value])

    #function analogous to predict of scikit-learn 
    def predict(self, X_test):
        Y_predicted = np.array([0 for i in range(len(X_test))])
        for i in range(len(X_test)):
            Y_predicted[i] = self.predict_data_point(X_test[i], self.root)
        return Y_predicted
    
    #function analogous to score of scikit-learn
    def score(self, X_test, Y_test):
        Y_predicted = self.predict(X_test)
        score = ((Y_predicted == Y_test).sum())/len(Y_test)
        return score

In [4]:
#function used to convert the decision tree to a dictionary
def make_dict(node, features, classes, dictionary):
    if(node.split_on_feature is None):
        return classes[node.current_output]
    else:
        dictionary[features[node.split_on_feature]] = {}
        for child in node.children.keys():
            res = make_dict(node.children[child], features, classes, {})
            dictionary[features[node.split_on_feature]][child] = res
    return dictionary

#function used to make the required graph of the decision tree using the dictionary created above
def make_graph(graph, dictionary, parent_node = None):
    for k in dictionary.keys():
        if parent_node is not None:
            from_name = parent_node.get_name().replace("\"", "") + '_' + str(k)
            from_label = str(k)

            node_from = pydot.Node(from_name, label = from_label)
            
            graph.add_node(node_from)
            graph.add_edge(pydot.Edge(parent_node, node_from))

            if (isinstance(dictionary[k], dict)):
                make_graph(graph, dictionary[k], node_from)

            else:
                to_name = str(k) + '_' + str(dictionary[k])
                to_label = str(dictionary[k])

                node_to = pydot.Node(to_name, label = to_label, shape = 'box')
                
                graph.add_node(node_to)
                graph.add_edge(pydot.Edge(node_from, node_to))

        else:
            from_name =  str(k)
            from_label = str(k)

            node_from = pydot.Node(from_name, label = from_label)
            
            graph.add_node(node_from)
            
            make_graph(graph, dictionary[k], node_from)

#function to plot the tree using the dictionary created above
def plot_tree(decision_tree, name):
    graph = pydot.Dot(graph_type = 'graph')
    make_graph(graph, decision_tree)
    graph.write_png(name + '.png')

# 1. Decision Tree for OR Operation

In [5]:
data = [[True, True, True], [False, True, True], [True, False, True], [False, False, False]]
df = pd.DataFrame(data, columns = ['X1', 'X2', 'Y'])

In [6]:
df

,X1,X2,Y
0,True,True,True
1,False,True,True
2,True,False,True
3,False,False,False


In [7]:
X_train = df.iloc[:, :-1]
Y_train = df.iloc[:, -1]
Y_train.replace({False: 0, True: 1}, inplace = True)

In [8]:
classifier = DecisionTreeClassifier()
classifier.fit(X_train.values, Y_train.values)
score = classifier.score(X_train.values, Y_train.values)
print('Accuracy:', score*100)

features = ['X1', 'X2']
classes = ['False', 'True']
decision_tree = make_dict(classifier.root, features, classes, {})
pprint(decision_tree)
plot_tree(decision_tree, 'OR')

Level 0
Count of 0 = 1
Count of 1 = 3
Current Entropy is = 0.8112781244591328
Splitting on feature X1 with gain ratio 0.31127812445913283

Level 1
Count of 0 = 1
Count of 1 = 1
Current Entropy is = 1.0
Splitting on feature X2 with gain ratio 1.0

Level 2
Count of 0 = 1
Count of 1 = 0
Current Entropy is = 0.0
Reached leaf Node

Level 2
Count of 0 = 0
Count of 1 = 1
Current Entropy is = 0.0
Reached leaf Node

Level 1
Count of 0 = 0
Count of 1 = 2
Current Entropy is =  0.0
Reached leaf Node

Accuracy: 100.0
{'X1': {False: {'X2': {False: 'False', True: 'True'}}, True: 'True'}}


# 2. Decision Tree for Play Tennis Dataset

In [14]:
df = pd.read_csv('play_tennis.csv')
df.drop('day', axis = 1, inplace = True)

X_train = df.iloc[:, :-1].values
Y_train = df.iloc[:, -1].replace({'Yes':1, 'No':0})

classifier = DecisionTreeClassifier()
classifier.fit(X_train, Y_train)

score = classifier.score(X_train, Y_train)
print('Accuracy:', score*100)

features = df.columns
classes = ['No', 'Yes']
decision_tree = make_dict(classifier.root, features, classes, {})
pprint(decision_tree)
plot_tree(decision_tree, 'play_tennis')

Level 0
Count of 0 = 5
Count of 1 = 9
Current Entropy is = 0.9402859586706311
Splitting on feature outlook with gain ratio 0.15642756242117528

Level 1
Count of 0 = 3
Count of 1 = 2
Current Entropy is = 0.9709505944546686
Splitting on feature humidity with gain ratio 1.0

Level 2
Count of 0 = 3
Count of 1 = 0
Current Entropy is =  0.0
Reached leaf Node

Level 2
Count of 0 = 0
Count of 1 = 2
Current Entropy is =  0.0
Reached leaf Node

Level 1
Count of 0 = 0
Count of 1 = 4
Current Entropy is =  0.0
Reached leaf Node

Level 1
Count of 0 = 2
Count of 1 = 3
Current Entropy is = 0.9709505944546686
Splitting on feature wind with gain ratio 1.0

Level 2
Count of 0 = 2
Count of 1 = 0
Current Entropy is =  0.0
Reached leaf Node

Level 2
Count of 0 = 0
Count of 1 = 3
Current Entropy is =  0.0
Reached leaf Node

Accuracy: 100.0
{'outlook': {'Overcast': 'Yes',
             'Rain': {'wind': {'Strong': 'No', 'Weak': 'Yes'}},
             'Sunny': {'humidity': {'High': 'No', 'Normal': 'Yes'}}}}


In [15]:
df

,outlook,temp,humidity,wind,play
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


# 3. Decision Tree for Iris Dataset

In [9]:
def ctc_util(value, *limits):
    if (limits[0] <= value < limits[1]):
        return '[minimum, (minimum + mean)/2)'
    elif (limits[1] <= value < limits[2]):
        return '[(minimum + mean)/2, mean)'
    elif (limits[2] <= value < limits[3]):
        return '[mean, (mean + maximum)/2)'
    elif (limits[3] <= value <= limits[4]):
        return '[(mean + maximum)/2, maximum]'

#function to convert the continuous values to categorical values
def continuous_to_categorical(df, feature_name):
    minimum = df[feature_name].min()
    mean = df[feature_name].mean()
    maximum = df[feature_name].max()
    
    #categories are assigned as follows
    #category1 = [b1, b2)
    #category2 = [b2, b3)
    #category3 = [b3, b4)
    #category4 = [b4, b5]
    b1 = minimum
    b2 = (minimum + mean)/2
    b3 = mean
    b4 = (mean + maximum)/2
    b5 = maximum
    
    return df[feature_name].apply(ctc_util, args = (b1, b2, b3, b4, b5))

In [10]:
iris = datasets.load_iris()
df = pd.DataFrame(iris.data)
df.columns = iris.feature_names

df[df.columns[0]] = continuous_to_categorical(df, df.columns[0])
df[df.columns[1]] = continuous_to_categorical(df, df.columns[1])
df[df.columns[2]] = continuous_to_categorical(df, df.columns[2])
df[df.columns[3]] = continuous_to_categorical(df, df.columns[3])

In [11]:
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,"[(minimum + mean)/2, mean)","[mean, (mean + maximum)/2)","[minimum, (minimum + mean)/2)","[minimum, (minimum + mean)/2)"
1,"[minimum, (minimum + mean)/2)","[(minimum + mean)/2, mean)","[minimum, (minimum + mean)/2)","[minimum, (minimum + mean)/2)"
2,"[minimum, (minimum + mean)/2)","[mean, (mean + maximum)/2)","[minimum, (minimum + mean)/2)","[minimum, (minimum + mean)/2)"
3,"[minimum, (minimum + mean)/2)","[mean, (mean + maximum)/2)","[minimum, (minimum + mean)/2)","[minimum, (minimum + mean)/2)"
4,"[minimum, (minimum + mean)/2)","[mean, (mean + maximum)/2)","[minimum, (minimum + mean)/2)","[minimum, (minimum + mean)/2)"
...,...,...,...,...
145,"[mean, (mean + maximum)/2)","[(minimum + mean)/2, mean)","[mean, (mean + maximum)/2)","[(mean + maximum)/2, maximum]"
146,"[mean, (mean + maximum)/2)","[minimum, (minimum + mean)/2)","[mean, (mean + maximum)/2)","[(mean + maximum)/2, maximum]"
147,"[mean, (mean + maximum)/2)","[(minimum + mean)/2, mean)","[mean, (mean + maximum)/2)","[(mean + maximum)/2, maximum]"
148,"[mean, (mean + maximum)/2)","[mean, (mean + maximum)/2)","[(mean + maximum)/2, maximum]","[(mean + maximum)/2, maximum]"


In [12]:
X = df.values
Y = iris.target
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 1)

classifier = DecisionTreeClassifier()
classifier.fit(X_train, Y_train)

score = classifier.score(X_test, Y_test)
print('Accuracy:', score*100)

features = iris.feature_names
classes = iris.target_names
decision_tree = make_dict(classifier.root, features, classes, {})
pprint(decision_tree)
plot_tree(decision_tree, 'iris')

Level 0
Count of 0 = 37
Count of 1 = 34
Count of 2 = 41
Current Entropy is = 1.5807197138422102
Splitting on feature petal length (cm) with gain ratio 0.680231787985011

Level 1
Count of 0 = 0
Count of 1 = 0
Count of 2 = 24
Current Entropy is =  0.0
Reached leaf Node

Level 1
Count of 0 = 37
Count of 1 = 0
Count of 2 = 0
Current Entropy is =  0.0
Reached leaf Node

Level 1
Count of 0 = 0
Count of 1 = 6
Count of 2 = 0
Current Entropy is =  0.0
Reached leaf Node

Level 1
Count of 0 = 0
Count of 1 = 28
Count of 2 = 17
Current Entropy is = 0.9564574047992594
Splitting on feature petal width (cm) with gain ratio 0.3518470770212301

Level 2
Count of 0 = 0
Count of 1 = 0
Count of 2 = 9
Current Entropy is =  0.0
Reached leaf Node

Level 2
Count of 0 = 0
Count of 1 = 3
Count of 2 = 0
Current Entropy is =  0.0
Reached leaf Node

Level 2
Count of 0 = 0
Count of 1 = 25
Count of 2 = 8
Current Entropy is = 0.7990485210442682
Splitting on feature sepal length (cm) with gain ratio 0.14662562110534788
